## Part 1

We need to create connect the closest junction boxes with wires and then check how many separate circuits we have when we made N connections. 

In [1]:
import io
from pathlib import Path

import helper as h
import networkx as nx
import numpy as np
import pandas as pd

logger = h.setup_logger("AoC_2025: Day 8")

In [2]:
def read_input(src: str | Path):
    if Path(src).is_file():
        with open(src, "r") as f:
            data = pd.read_csv(f, header=None, index_col=False).to_numpy()
    else:
        data = pd.read_csv(io.StringIO(src), header=None, index_col=False).to_numpy()

    return data


In [3]:
def get_n_elements(arr, top_n_edges=1000):
    G = nx.Graph()
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            dist_ = np.linalg.norm(arr[i] - arr[j])
            G.add_edge(i, j, weight=dist_)
    

    # sort edges
    sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]["weight"])
    filtered_edges = sorted_edges[:top_n_edges]

    # create the new graph where you filter
    filtered_graph = nx.Graph()
    filtered_graph.add_edges_from([(u, v, data) for u, v, data in filtered_edges])
    components = list(nx.connected_components(filtered_graph))

    return components


def part1(src, top_n_edges=1000):
    a = read_input(src)
    circuits = get_n_elements(a, top_n_edges)
    size_circuits = sorted([len(circuit) for circuit in circuits], reverse=True)
    return size_circuits[0] * size_circuits[1] * size_circuits[2]

h.extended_assert("toy", part1("inputs/day_08_toy.txt", top_n_edges=10), 40)

In [4]:
part1("inputs/day_08.txt")

68112

## Part 2

In part two, the question is how many connections we need to make to have all junction boxes connected in a single circuit.

In [5]:
def last_two(arr):
    G = nx.Graph()
    # add all nodes:
    G.add_nodes_from(range(len(arr)))

    dist_dict = {}
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            dist_ = np.linalg.norm(arr[i] - arr[j])
            if dist_dict.get(dist_) is not None:
                raise KeyError("repeating distances")
            dist_dict[dist_] = (i, j)
    
    for dist_ in sorted(dist_dict.keys()):
        G.add_edges_from([dist_dict[dist_]])
        if nx.is_connected(G):
            return (arr[dist_dict[dist_][0]], arr[dist_dict[dist_][1]])

    raise ValueError("graph not connected")


In [6]:
def part2(src):
    a = read_input(src)
    jbox1, jbox2 = last_two(a)
    return jbox1[0] * jbox2[0]

h.extended_assert("toy", part2("inputs/day_08_toy.txt"), 25272)

In [7]:
part2("inputs/day_08.txt")

np.int64(44543856)